# Main 

## Initialisation 

### Import des librairies

In [5]:
import pandas as pd # Pour manipuler des dataframes
import numpy as np # Pour manipuler des arrays
import matplotlib.pyplot as plt # Pour afficher des graphiques
import seaborn as sns # Pour améliorer l'affichage des graphiques

### Import des données

In [ ]:
# Paramètres
nbLignes = 1000

# noms fichiers
donneesOctobre2019 = "2019-Oct.csv.gz"

# Chargement des données
data = pd.read_csv(donneesOctobre2019, nrows=nbLignes)

print(data.head(2))